In [116]:
import pandas as pd
import plotly.express as px
import ipywidgets as ipw
from IPython.display import display


#  vs code muestra los gráficos de plotly de forma nativa
#  Jupyter Notebook del explorador no muestra los gráficos de plotly de forma nativa
# Descomentar lo siguiente si se quiere usar Plotly en Jupyter Notebook del explorador

import plotly.io as pio

In [117]:
## cargado de tablas hechas por AnalisisOfertas.ipynb
chile = pd.read_csv("regiones-chile.csv", sep=";",encoding="utf-8")
tabla_puestos_comunas_total = pd.read_csv("../data/tabla_puestos_comunas_total.csv", sep=",", encoding="utf-8")
promedio_salario_por_puesto = pd.read_csv("../data/promedio_salarios_por_puesto.csv", sep=",", encoding="utf-8")
promedio_salarios_total = pd.read_csv("../data/promedio_salarios_por_ciudad_comuna.csv", sep=",", encoding="utf-8")
tabla_ciudades_total = pd.read_csv("../data/total_ofertas_por_ciudad.csv", sep=",", encoding="utf-8")
tabla_puestos_comunas_salarios = pd.read_csv("../data/tabla_puestos_comunas_salario.csv", sep=",", encoding="utf-8")


# Rehacer gráfico de barras de ofertas por región por error en la anterior iteracion
# Crear un diccionario para mapear comuna a región
comuna_a_region = chile.set_index('Comuna')['Región'].to_dict()

# Clasificar cada ciudad/comuna y sumar la cantidad de ofertas por región
tabla_ciudades_total['Región'] = tabla_ciudades_total['Ciudad/Comuna'].map(lambda x: comuna_a_region.get(x.lower(), comuna_a_region.get(x.capitalize(), None)))

# Agrupar por región y sumar las ofertas
ofertas_por_region = (
    tabla_ciudades_total.groupby('Región', as_index=False)['Cantidad_Ofertas']
    .sum()
    .sort_values('Región')
    .reset_index(drop=True)
)


In [118]:
def grafico_iteract_ofertas_por_ciudad(comuna, plot_output=None):  
    # Filtrar los datos para la comuna seleccionada
    datos = tabla_puestos_comunas_total[tabla_puestos_comunas_total['Comuna'].str.lower() == comuna.lower()]
    if datos.empty:
        print(f"No hay datos para la comuna '{comuna}'.")
        return
    fig = px.pie(
        datos,
        names="Puesto",
        values="Cantidad_Ofertas",
        title=f"Distribución de Puestos en {comuna.capitalize()}",
        hole=0.4,
        custom_data=["Puesto", "Cantidad_Ofertas"]
    )
    fig.update_traces(
        textinfo='label',
        hovertemplate='<b>Puesto:</b> %{label}<br><b>Cantidad de ofertas:</b> %{value}<extra></extra>'
    )
    fig.update_layout(
        transition=dict(
            duration=500,  # milisegundos
            easing="cubic-in-out"
        )
    )

    if plot_output is not None:
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    else:
        fig.show()

In [119]:
def grafico_ofertas_por_region_interactivo(ofertas_por_region=ofertas_por_region, plot_output=None):
    fig = px.bar(
        ofertas_por_region,
        x="Región",
        y="Cantidad_Ofertas",
        labels={'Región': 'Región', 'Cantidad_Ofertas': 'Cantidad de Ofertas'},
        title='Cantidad de Ofertas Laborales por Región',
        color_discrete_sequence=['skyblue']
    )
    fig.update_layout(
        xaxis_title='Región',
        yaxis_title='Cantidad de Ofertas',
        transition=dict(
            duration=500,
            easing="cubic-in-out"
        )
    )
    fig.update_traces(
        hovertemplate='<b>Región:</b> %{x}<br><b>Cantidad de ofertas:</b> %{y}<extra></extra>'
    )
    fig.update_yaxes(type='log')
    if plot_output is not None:
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    else:
        fig.show()


In [120]:
def grafico_salarios_por_puesto_comuna_interactivo(puesto, comuna, tabla_puestos_comunas_salarios=tabla_puestos_comunas_salarios, plot_output=None):
    # Filtrar los datos por puesto y comuna
    datos = tabla_puestos_comunas_salarios[
        (tabla_puestos_comunas_salarios['Puesto'].str.lower() == puesto.lower()) &
        (tabla_puestos_comunas_salarios['Ubicacion'].str.lower() == comuna.lower())
    ]
    if datos.empty:
        print(f"No hay datos de salarios para el puesto '{puesto}' en la comuna '{comuna}'.")
        return

    # Ordenar los datos por salario para mostrar la tendencia
    datos_sorted = datos.sort_values("Salario").reset_index(drop=True)
    datos_sorted["Oferta"] = datos_sorted.index + 1  # Para eje x

    # Formatear el salario con puntos de miles y símbolo $
    def formatear_salario(salario):
        return f"${salario:,.0f}".replace(",", ".")
    datos_sorted["SalarioFormateado"] = datos_sorted["Salario"].apply(formatear_salario)

    fig = px.bar(
        datos_sorted,
        x="Oferta",
        y="Salario",
        text="SalarioFormateado",
        title=f"Ofertas con salarios informados para '{puesto}' en {comuna.capitalize()}",
        labels={"Oferta": "Oferta", "Salario": "Salario CLP"},
        color_discrete_sequence=['#1f77b4']
    )
    fig.update_layout(
        xaxis_title="Oferta (ordenadas por salario)",
        yaxis_title="Salario CLP",
        transition=dict(duration=500, easing="cubic-in-out")
    )
    fig.update_xaxes(tickmode='linear', dtick=1)
    fig.update_traces(
        hovertemplate='<b>Oferta:</b> %{x}<br><b>Salario:</b> %{text}<extra></extra>',
        textposition='outside'
    )

    if plot_output is not None:
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    else:
        fig.show()


In [121]:
def graficar_tendencia_salario_por_puesto(puesto, df=tabla_puestos_comunas_salarios, output=None):
    datos = df[df["Puesto"].str.lower() == puesto.lower()]
    
    if datos.empty:
        with output:
            output.clear_output()
            display(ipw.HTML(f"<b>No hay datos para el puesto '{puesto}'</b>"))
        return

    # Ordenamos por salario de mayor a menor
    datos = datos.sort_values(by="Salario", ascending=False)

    fig = px.bar(
        datos,
        x="Ubicacion",
        y="Salario",
        title=f"Salario Promedio por Comuna - Puesto: {puesto}",
        color="Salario",
        color_continuous_scale="Blues",
        text_auto=True
    )

    fig.update_layout(
        xaxis_title="Comuna",
        yaxis_title="Salario Promedio",
        transition_duration=500
    )

    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Salario promedio: $%{y:,.0f}<extra></extra>"
    )
    with output:
        output.clear_output(wait=True)
        fig.show()


In [122]:
def graficar_salarios_promedio_por_ciudad(df=promedio_salarios_total,output=None):
    fig = px.bar(
        df.sort_values("Promedio_Salario", ascending=False),
        x="Ciudad/Comuna",
        y="Promedio_Salario",
        title="Salario Promedio por Comuna",
        labels={"Ciudad/Comuna": "Comuna", "Promedio_Salario": "Salario Promedio CLP"},
        color="Promedio_Salario",
        color_continuous_scale="Blues_r"  # Cambia la escala aquí
    )
    fig.update_layout(
        xaxis_title="Comuna",
        yaxis_title="Salario Promedio CLP",
        xaxis_tickangle=-45
    )
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>Salario promedio: $%{y:,.0f}<extra></extra>"
    )
    with output:
        output.clear_output(wait=True)
        fig.show()

In [127]:

region_dropdown = ipw.Dropdown(description="Región:", layout=ipw.Layout(width="300px")) #type: ignore
provincia_dropdown = ipw.Dropdown(description="Provincia:", layout=ipw.Layout(width="300px"))# type: ignore
comuna_dropdown = ipw.Dropdown(description="Comuna:", layout=ipw.Layout(width="300px"))# type: ignore
puesto_dropdown = ipw.Dropdown(description="Puesto:", layout=ipw.Layout(width="300px"))# type: ignore

boton_tendencia = ipw.Button(
    description='Ver tendencia salarial en otras comunas',
    button_style='info',
    layout=ipw.Layout(width='auto', visibility='hidden')  # Oculto por defecto
)

boton_salarios_total = ipw.Button(
    description='Ver Salarios Promedio por comunas',
    button_style='info',
    layout=ipw.Layout(width='auto', visibility='visible')  # Oculto por defecto
)

boton_reset = ipw.Button(
    description="🔄 Resetear filtros",
    button_style="warning",
    layout=ipw.Layout(width="220px" ,margin="10px 0 0 80px") # type: ignore
)


rango_dropdown = ipw.IntSlider( # type: ignore
    value=0,
    min=0,
    max=200000000,
    step=1,
    description='Salario:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

puesto_dropdown = ipw.Dropdown(  # type: ignore
    options=["Ninguno"] + sorted(promedio_salario_por_puesto["Puesto"].unique()), # type: ignore
    description='Puesto:',
    layout=ipw.Layout(width='300px') # type: ignore
)

active_tendencia = False
active_salarios = False   
    

# -------- KPIs --------
kpi_empleos = ipw.HTML(value="<h3 style='text-align:center;'>100<br><span style='font-size:14px'>Cantidad total de empleos</span></h3>") # type: ignore
kpi_salario = ipw.HTML(value="<h3 style='text-align:center;'>$1.200.000<br><span style='font-size:14px'>Salario Promedio</span></h3>") # type: ignore

kpi_box_1 = ipw.HBox([ipw.Label("🔢"), kpi_empleos]) # type: ignore
kpi_box_2 = ipw.HBox([ipw.Label("💰"), kpi_salario]) # type: ignore
kpi_section = ipw.HBox([kpi_box_1, kpi_box_2])# type: ignore

# -------- Imagen o gráfico placeholder --------
plot_output = ipw.Output()# type: ignore
with plot_output:
    plot_output.clear_output(wait=True)
    grafico_ofertas_por_region_interactivo(plot_output=plot_output)# type: ignore


def actualizar_kpis():
    # Si hay una comuna seleccionada, filtrar por comuna
    comuna = comuna_dropdown.value
    if comuna:
        # Filtrar tabla_ciudades_total por la comuna seleccionada
        filtro = tabla_ciudades_total["Ciudad/Comuna"].str.lower() == comuna.lower()# type: ignore
        total = tabla_ciudades_total.loc[filtro, "Cantidad_Ofertas"].sum()  # type: ignore
        # filtrar promedio_salarios_total por la comuna seleccionada
        filtro_salario = promedio_salarios_total["Ciudad/Comuna"].str.lower() == comuna.lower() # type: ignore
        # Calcular el promedio de salario solo para la comuna seleccionada
        promedio = promedio_salarios_total.loc[filtro_salario, "Promedio_Salario"].sum() # type: ignore
    else:
        total = tabla_ciudades_total["Cantidad_Ofertas"].sum()# type: ignore
        promedio = promedio_salarios_total["Promedio_Salario"].mean()  # type: ignore

    kpi_empleos.value = f"<h3 style='text-align:center;'>{int(total)}<br><span style='font-size:14px'>Cantidad total de empleos</span></h3>"
    kpi_salario.value = f"<h3 style='text-align:center;'>${promedio:,.0f}<br><span style='font-size:14px'>Salario Promedio</span></h3>"
    # -------- Layout general por pestaña --------   
    return None


##
def actualizar_provincias(change):        
    region = change['new']
    provincias = chile[chile["Región"] == region]["Provincia"].unique()# type: ignore
    provincia_dropdown.options = sorted(provincias)
    
    # Resetear comunas
    comuna_dropdown.options = []

def actualizar_comunas(change):
    provincia = change['new']
    comunas = chile[chile["Provincia"] == provincia]["Comuna"].unique()# type: ignore
    comuna_dropdown.options = sorted(comunas)


def tab_1():
    filtros = ipw.VBox([region_dropdown, provincia_dropdown, comuna_dropdown, puesto_dropdown,boton_reset])# type: ignore
    derecha = ipw.VBox([# type: ignore
            plot_output,boton_tendencia,boton_salarios_total
    ], layout=ipw.Layout(width='600px', min_width='400px', max_width='800px', align_items='flex-start'))# type: ignore
    layout = ipw.HBox([# type: ignore
        filtros,
        ipw.VBox([kpi_section, derecha], layout=ipw.Layout(margin='0 0 0 40px'))# type: ignore
    ])
    return layout

##
region_dropdown.options = sorted(chile["Región"].unique())# type: ignore
provincia_dropdown.options = []
comuna_dropdown.options = []
actualizar_kpis()  # Inicializar KPIs al cargar la pestaña

region_dropdown.observe(actualizar_provincias, names='value')
provincia_dropdown.observe(actualizar_comunas, names='value')
def on_comuna_change(change):
    comuna = change['new']
    actualizar_kpis()  # Actualizar KPIs al cambiar la comuna
    if comuna:
        boton_salarios_total.layout.visibility = 'hidden'
        with plot_output:
            plot_output.clear_output(wait=True)
            grafico_iteract_ofertas_por_ciudad(comuna, plot_output=plot_output)# type: ignore
        


def on_change_puesto(change):
    puesto = change['new']
    comuna = comuna_dropdown.value
    global active_tendencia

    if puesto != "Ninguno" and comuna:
        if active_tendencia:
            boton_tendencia.description = "Ver tendencia salarial por comuna"
            active_tendencia = False
        
        with plot_output:
            plot_output.clear_output(wait=True)
            grafico_salarios_por_puesto_comuna_interactivo(
                puesto, comuna, tabla_puestos_comunas_salarios,
                plot_output=plot_output
            )
        boton_tendencia.layout.visibility = 'visible'
    else:
        boton_tendencia.layout.visibility = 'hidden'
        

def on_boton_tendencia_clicked(b):
    global active_tendencia
    puesto = puesto_dropdown.value
    puesto = puesto_dropdown.value
    comuna = comuna_dropdown.value

    if puesto != "Ninguno" and comuna:
        with plot_output:
            plot_output.clear_output(wait=True)

            if not active_tendencia:
                # Mostrar gráfico de tendencia
                graficar_tendencia_salario_por_puesto(puesto,output=plot_output)
                b.description = "Volver al gráfico de ofertas por comuna"
                active_tendencia = True

            else:
                # Volver al gráfico original por comuna
                grafico_salarios_por_puesto_comuna_interactivo(
                    puesto, comuna, tabla_puestos_comunas_salarios, plot_output
                )
                b.description = "Ver tendencia salarial por comuna"
                active_tendencia = False
     

def on_boton_salarios_clicked(b):
    global active_salarios
    with plot_output:
            plot_output.clear_output(wait=True)
            if not active_salarios:
                # Mostrar gráfico de tendencia
                graficar_salarios_promedio_por_ciudad(output=plot_output)
                b.description = "Volver al gráfico Ofertas por Región"
                active_salarios = True

            else:
                # Volver al gráfico original por comuna
                grafico_ofertas_por_region_interactivo(ofertas_por_region=ofertas_por_region, plot_output=plot_output)
                b.description = "Ver Salarios Promedio por comunas"
                active_salarios = False 
                    
def resetear_filtros(b):
    # Limpiar valores de widgets
    region_dropdown.value = None
    provincia_dropdown.options = []
    provincia_dropdown.value = None
    comuna_dropdown.options = []
    comuna_dropdown.value = None
    puesto_dropdown.value = "Ninguno"
    rango_dropdown.value = 0

    # Restaurar gráfico general
    global active_salarios
    active_salarios = False
    with plot_output:
        plot_output.clear_output(wait=True)
        grafico_ofertas_por_region_interactivo(plot_output=plot_output)

    # Restaurar KPIs
    actualizar_kpis()

    # Resetear botón de tendencia
    boton_tendencia.layout.visibility = "hidden"
    boton_tendencia.description = "Ver tendencia salarial por comuna"
    boton_salarios_total.layout.visibility = "visible"
    boton_salarios_total.description = "Ver Salarios Promedio por comunas"

    # Reiniciar flag de gráfico
    global mostrar_tendencia
    mostrar_tendencia = False


boton_reset.on_click(resetear_filtros)
boton_tendencia.on_click(on_boton_tendencia_clicked)
boton_salarios_total.on_click(on_boton_salarios_clicked)
comuna_dropdown.observe(on_comuna_change, names='value')
puesto_dropdown.observe(on_change_puesto, names='value')# type: ignore


# Otras pestañas vacías por ahora
tab_2 = ipw.HTML("<h3>Tecnologías (en desarrollo)</h3>")# type: ignore
tab_3 = ipw.HTML("<h3>Seguridad (en desarrollo)</h3>")# type: ignore

# -------- Tabs dashboard --------
tabs = ipw.Tab()# type: ignore
tabs.children = [tab_1(), tab_2, tab_3]
tabs.set_title(0, "Cantidad de empleos y salarios")
tabs.set_title(1, "Tecnologías")
tabs.set_title(2, "Seguridad")

display(tabs)# type: ignore